<a href="https://colab.research.google.com/github/sergioaugusto94/Recozimento/blob/main/Recozimento_model_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo Recozimento

O algoritmo desenvolvido busca prever a qualidade do processo de tratamento térmico do recozimento. É necessário ter mais detalhes do modelo de negócio, mas a principio, o algoritmo foi elaborado de forma a minimizar a quantidade de recozimentos ideais que são classificados como ruins (no caso, a peça é descartada incorretamente, o que gera redução do lucro da companhia) e também minimizar a quantidade de recozimentos ruins que são classificados como bons (nesse caso, a peça é tratada como conforme e pode falar em trabalho, o que pode acarretar prejuizos à imagem da empresa). 

O processo de tratamento de dados foi uma das etapas que mostrou melhores avanços no aprendizado final do algoritmo. A performance dos algoritmos foi testada a cada processo de tratamento de dados. Nesse trabalho, foram utilizados 4 processos de tratamento de dados: 
obs: Em todas as etapas foi utilizada a imputação por regressão.

**1- Imputação com correlação das colunas + Valor de corte para valores faltantes = 600 + Valor de corete para valores nulos = 200**
    A lógica por detrás dessa etapa foi que colunas com muitos dados faltantes ou nulos não seriam relevantes no aprendizado do algoritmo. Por isso essas foram descartadas. No entanto, como pode se observar no gráfico abaixo, o resultado de descartar muitas colunas resultou em um aprendizado muito fraco. 

**2- Imputação com correlação das colunas + Valor de corte para valores faltantes = 700**
    Nesse processo de tratamento, foram utilizadas mais colunas que no processo anterior, já que se permitiram usar colunas com mais de 700 dados faltantes como input no aprendizado do algoritmo. Essa abordagem trouxe ganhos consideráveis no aprendizado. 
    
**3- Imputação com correlação das colunas + Valor de corte para valores faltantes = 750 + Relação 'temper rolling' e recozimento 'ideal'**
    Já nesse processo, aumentou-se ainda mais as colunas para input no algoritmo, bem como houve um tratamento especial da coluna 'temper_rolling', já que esta última possuia uma correlação muito boa com o recozimento ideal, o que poderia ajudar no aprendizado do algoritmo. Apesar da alta correlação, esse processo trouxe ganhos de aprendizado mínimos. 
        
**4- Processo 3 + Técnica de Subamostragem**
    Nesse procedimento, tentou-se minimizar os registros das classes que mais aparecem na base de dados. Essa etapa mostrou-se bastante promissora, na qual alguns algoritmos atingiram scores superiores a 90%.
    
Cada um dos processos é explicado mais a fundo ao longo do trabalho. A imagem abaixo traz a evolução dos algoritmos em função do processo de tratamento de dados usado. 
    
![image.png](attachment:image.png)

**Tratamento da base de dados**

A primeira etapa do tratamento de dados é a importação dos dados a partir das planilhas csv.
Foram unidas as planilhas com a data de realização do experimento e os dados de cada experimento. 
Após essa etapa, foram filtrados os experimentos realizados em agosto de 2020.

In [34]:

import pandas as pd
import numpy as np
import pickle

date = pd.read_csv('data_experimentos.csv')
dataset = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

dataset= pd.concat([date,dataset], axis=1, join='inner')

#Filtrando os dados
dataset = dataset.loc[dataset['ano']==2020]
dataset = dataset.loc[dataset['mes']==8]



**Remoção de campos que não são relevantes para o algoritmo**

1- A coluna 'product-type' possui um único valor, logo não tem capacidade de aprimorar o aprendizado do algoritmo.

2- As colunas 'experimento', 'ano' e 'mes' não tem relação com a qualidade do recozimento. 


In [35]:
#%% Valores unicos na coluna/ Não relevantes para o algoritmo
dataset.loc[dataset['product-type'].notnull(),'product-type'].describe()
dataset = dataset.drop(columns = ['product-type','experimento','ano','mes']) 

**Remoção de colunas com muitos valores faltantes/ muito valores 0.**

1- Muitos valores faltantes prejudicam/ não influenciam na aprendizagem do algoritmo. Foram feitos vários testes do valor de corte de uma determinada coluna. Caso uma coluna tenha um número de registros vázios acima desse limite, a coluna é deletada da base de dados. Foram feitos vários testes para estipular o melhor valor de corte:
    
    a) Valor de corte 600
    
    b) Valor de corte 700
    
    c) Valor de corte 750

2- Muitos valores 0 impoem uma tendência indezejada no algoritmo. Foi estabelecido que uma coluna que tinha menos de 200 valores não nulos seria eliminada. 

obs: 

    . Foi realizado o tratamento dos dados faltantes, que na base de dados eram registrados como '?'

    . Os dados 'formability' eram tratados como string e estes foram convertidos para float
    
No código é salvada as colunas trabalhadas para ser utilizada posteriormente no script do Python

In [36]:
# Valores Faltantes
dataset.replace('?', np.nan, inplace=True)
dataset['formability'] = dataset['formability'].astype(float)
dataset.isnull().sum()

colunas_faltantes = []
for i in range(len(dataset.columns.values)):
    if dataset.isnull().sum()[i] > 750:
        colunas_faltantes.append(dataset.columns.values[i])

# Muitos Zeros
colunas_zeros = []
# for coluna in dataset.columns.values:
#     if np.count_nonzero(dataset[coluna]) < 200:
#         colunas_zeros.append(coluna)

pickle.dump(colunas_faltantes,open('colunas_faltantes.txt','wb'))
        
dataset = dataset.drop(columns = colunas_faltantes+colunas_zeros)

Foi verificado que existe uma correlação muito boa entre os valores da coluna 'temper_rolling' e o recozimento classificado como 'ideal', como observado na figura abaixo: 
![image-2.png](attachment:image-2.png)

Dessa forma, para evitar que essa correlação seja perdida nas etapas posteriores do tratamento de dados, na qual vamos fazer a imputação de valores por meio da técnica de regressão, vamos preencher os registros vazios dessa coluna com o valor 'NULA'.

O mesmo foi feito com as colunas 'bf', 'bl', 'oil', 'cbond' e 'bt' para verificar se há algum benefício na performance do algoritmo em utilizar essas técnicas no tramento de dados. 

In [37]:
# Foi observado um padrão forte entre a classe 'Ideal' com a coluna 'Temper_rolling'

dataset.loc[dataset['temper_rolling'].isnull(),'temper_rolling'] = 'NULA'
dataset.loc[dataset['bf'].isnull(),'bf'] = 'NULA'
dataset.loc[dataset['bl'].isnull(),'bl'] = 'NULA'
dataset.loc[dataset['oil'].isnull(),'oil'] = 'NULA'
dataset.loc[dataset['cbond'].isnull(),'cbond'] = 'NULA'
dataset.loc[dataset['bt'].isnull(),'bt'] = 'NULA'



Correção das colunas 'thick','width' e 'len' para que o algoritmo entenda a variável como um float

In [38]:
cor_strings = ['thick','width','len']
for coluna in cor_strings:
    for i in range(len(dataset)):
        dataset.loc[i,coluna]=float(dataset[coluna][i].replace('_',''))
        

Pegando a localização de cada uma das colunas que possuem variáveis categóricas para realização do Label Encoder

In [39]:
dataset_notnull = pd.DataFrame(dataset.dropna())

coluna_string = []
dataset_notnull.loc[0,:]=dataset.iloc[0,:].values
for i in range(dataset_notnull.shape[1]):
    if type(dataset_notnull.iloc[0,:][i])==str:
        coluna_string.append(i)

Fazendo o Label Encoder, para converter as variáveis categóricas em variáveis discretas

In [40]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

dataset_notnull=dataset_notnull.iloc[:,:].values

for i in coluna_string:
    dataset_notnull[:,i]=label_encoder.fit_transform(dataset_notnull[:,i])

dataset_notnull = pd.DataFrame(dataset_notnull,columns=dataset.columns.values,dtype = 'float64')


**Imputação de dados pela alta correlação entre as colunas**

Existe uma grande correlação entre as colunas 'surface-quality', 'condition', 'family' e 'steel', por esse motivo, optou-se pelo preenchimento dos valores faltantes conforme o código abaixo. Esse procedimento seguiu a lógica mostrada na figura abaixo, na qual percebemos que quando 'surface-quality' é 'E', a maior parte de 'conditions' é 'S'. Dessa forma, toda a coluna conditions é preenchida com 'S'.
![image.png](attachment:image.png)

In [41]:
# Existe uma grande correlação entre Surface Quality, Condition e Steel
dataset.loc[dataset['surface-quality']=='E','condition']='S' 
dataset.loc[dataset['surface-quality']=='D','condition']='S' 
dataset.loc[dataset['surface-quality']=='F','condition']='S'
dataset.loc[dataset['condition']=='A','surface-quality']='G' 
dataset.loc[dataset['steel']=='A','condition']='S' 
dataset.loc[dataset['steel']=='M','surface-quality']='G' 
dataset.loc[(dataset['surface-quality']=='G') & (dataset['condition']=='A'),'steel']='R'
dataset.loc[(dataset['steel']=='R') & (dataset['condition']=='S'),'surface-quality']='E'
dataset.loc[dataset['family']=='TN','steel']='A'

dataset.isnull().sum()

exp_id               0
family             691
steel               50
carbon               0
hardness             0
temper_rolling       0
condition          121
formability        291
strength             0
non-ageing         713
surface-quality    136
bf                   0
bt                   0
bw/me              617
bl                   0
cbond                0
shape                0
thick                0
width                0
len                  0
oil                  0
bore                 0
recozimento          0
dtype: int64

Mesmo preenchendo os registros com o método acima, ainda assim, algumas colunas possuiam alguns valores vazios. Para resolver esse problema, utilizou-se a técnica de imputação de valores a partir da Regressão Linear Multipla.

**Imputação por Regressão Linear Multipla**

1- O primeiro passo para implementar essa técnica é localizar as colunas que possuem valores faltantes. 

2- Logo após, definimos a função que adiciona ao DataFrame de trabalho colunas de imputação, na qual somente a linha com valores faltantes é preenchida com valores aleatórios da coluna em questão.

3- Com a função definida, cada coluna que possui valores faltantes é chamada em um loop. As colunas criadas, com o final '_imputation', são copiadas das colunas originais. Logo após a função do item anterior é chamada, que fará o preenchimento dos registros faltantes com valores aleatórios. No mesmo loop os valores categóricos são transformados em variáveis discretas, através do Label Encoder. 

4- No loop seguinte, fazemos a substituição dos valores contidos nas colunas originais da base de dados, pelos valores contidos nas colunas de final '_imputation'. Ao mesmo tempo, fazemos o procedimento de Label Encoder nas variáveis categóricas da coluna original. 

5- Em seguida, criamos um novo DataFrame, que irá armazenar os valores calculados pela regressão linear. Nesse modelo de regressão, ele utiliza todas as colunas como entrada, com excessão da coluna 'recozimento' (para não haver nenhuma influência na hora de realizar o treinamento dos algoritmos) para realizar o cálculo dos valores faltantes. 

In [42]:
colunas_faltantes = []
for i in range(len(dataset.isnull().sum())):
    if dataset.isnull().sum()[i]>1:
        colunas_faltantes.append(dataset.isnull().sum().index[i])

def random_input(df,coluna):
    numeros_faltantes = df[coluna].isnull().sum()
    valores_observados = df.loc[df[coluna].notnull(),coluna]
    df.loc[df[coluna].isnull(),coluna+'_imputation']=np.random.choice(valores_observados,numeros_faltantes,replace=True)
    #cria uma coluna no DF, com valores aleatorios de idade, onde não há o valor de idade
    return df

for coluna in colunas_faltantes:
    encoded=dataset.iloc[:,dataset.columns.get_loc(coluna)].values
    dataset[coluna+'_imputation'] = dataset[coluna]
    dataset = random_input(dataset,coluna)
    if coluna != 'formability':
        encoded=label_encoder.fit_transform(dataset[coluna+'_imputation'].values)
        dataset[coluna+'_imputation'] = dataset[coluna+'_imputation'].replace(dataset[coluna+'_imputation'].values.tolist(),encoded)

for coluna in dataset.columns.values:
    if colunas_faltantes.count(coluna)>0:
        encoded = dataset.iloc[:,dataset.columns.get_loc(coluna+'_imputation')].values
        dataset[coluna] = dataset[coluna].replace(dataset[coluna].values.tolist(),encoded)
    if type(dataset[coluna][0]) == str or coluna == 'family':
        encoded=label_encoder.fit_transform(dataset[coluna].values)
        dataset[coluna] = dataset[coluna].replace(dataset[coluna].values.tolist(),encoded)
    
from sklearn.linear_model import LinearRegression
deter_data = pd.DataFrame(columns = ['Det' + coluna for coluna in colunas_faltantes])
for coluna in colunas_faltantes:
    deter_data['Det'+coluna]=dataset[coluna+'_imputation']
    parameters = list(set(dataset.columns)-set(colunas_faltantes)-{coluna+'_imputation'}-{'recozimento'})

    model = LinearRegression()
    model.fit(X=dataset[parameters],y=dataset[coluna+'_imputation'])
    
    deter_data.loc[dataset[coluna].isnull(),'Det'+coluna] = model.predict(dataset[parameters])[dataset[coluna].isnull()]
    dataset[coluna]=dataset[coluna+'_imputation']
    dataset = dataset.drop(columns=coluna+'_imputation')

Fazemos agora o processamento final dos dados, separando os dados de input para o algoritmo:

O One Hot Encoder não mostrou ganhos significativos na aprendizagem do algoritmo. Por esse motivo ele foi desconsiderado no script final, pois compromete o processamento dos dados. 

In [43]:
x_data = dataset.iloc[:,1:dataset.shape[1]-1].values

    ##--One Hot Encoder
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
 
# onehotencoder_data=ColumnTransformer(transformers=[('OneHot',OneHotEncoder(),[dataset.columns.get_loc("steel"),dataset.columns.get_loc("surface-quality")])],remainder='passthrough')
# x_data = onehotencoder_data.fit_transform(x_data)

    ##--Escalonamento dos dados
from sklearn.preprocessing import StandardScaler 
scaler_data = StandardScaler()
x_data = scaler_data.fit_transform(x_data)


Aplicando o Label Encoder nas classes: 

In [44]:
encoded=label_encoder.fit_transform(dataset['recozimento'].values)
dataset['recozimento'] = dataset['recozimento'].replace(dataset['recozimento'].values.tolist(),encoded)


y_data = dataset.iloc[:,dataset.shape[1]-1].values

**Subamostragem**

Como nossa base de dados está um pouco desbalanceada, foi aplicada a técnica de subamostragem Tomek Links de forma a amenizar a componente de tendencia para a classe que mais aparece no banco de dados. 

In [45]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks(sampling_strategy='all') 
x_data, y_data = tl.fit_sample(x_data,y_data)

np.unique(y_data,return_counts=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(array([0, 1, 2]), array([155, 138, 409]))

**Tuning dos Algoritmos**

Para o caso, foram utilizados os seguintes algoritmos: Random Forest, KNN, SVM, Redes Neurais e Regressão Logistica. 
O tuning desses algoritmos foi realizado usando o GridSearch.
Ao fim do Tuning, foram obtidos os valores dos melhores parâmetros e também o melhor score da combinação dos algoritmos. 

In [46]:
#%% TUNNIG DOS ALGORITMOS
from sklearn.model_selection import GridSearchCV
 
    # -----Random Forest
from sklearn.ensemble import RandomForestClassifier
# parametros1 = {'criterion': ['gini','entropy'],
#               'n_estimators': [23,14,21],
#               'min_samples_split':[3,7,9,5],
#               'min_samples_leaf':[1,3,7,2]}
# grid_search = GridSearchCV(estimator=RandomForestClassifier(),param_grid=parametros1)
# grid_search.fit(x_data,y_data)
# melhor_parametro = grid_search.best_params_
# melhor_resultado = grid_search.best_score_
# print(melhor_parametro)
# print(melhor_resultado) 

    #-----KNN
from sklearn.neighbors import KNeighborsClassifier
# parametros1 = {'n_neighbors': [2,3,5,9,10,11,12,13,14],
#               'p': [1,2], 'weights': ['uniform', 'distance'] }
# grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parametros1)
# grid_search.fit(x_data,y_data)
# melhor_parametro = grid_search.best_params_
# melhor_resultado = grid_search.best_score_
# print(melhor_parametro)
# print(melhor_resultado)

    #-----SVM
from sklearn.svm import SVC
# parametros1 = {'tol': [0.001,0.0001,0.00001],
#               'C': [2.4,2.0,1.8],
#               'kernel': ['rbf','linear','poly'],
#               }
# search_grid = GridSearchCV(estimator=SVC(), param_grid=parametros1)
# search_grid.fit(x_data,y_data)
# melhor_parametro = search_grid.best_params_
# melhor_resultado = search_grid.best_score_
# print(melhor_parametro) 
# print(melhor_resultado) 

    #----Redes Neurais
from sklearn.neural_network import MLPClassifier
# parametros1 = {'activation': ['relu','logistic','tanh'],
#               'solver': ['adam','sgd'],
#               'batch_size': [10,56],
#               'hidden_layer_sizes': [(5,5),(5,5,5)]} 
# search_grid = GridSearchCV(estimator=MLPClassifier(), param_grid=parametros1)
# search_grid.fit(x_data,y_data)
# melhor_parametro = search_grid.best_params_
# melhor_resultado = search_grid.best_score_
# print(melhor_parametro) 
# print(melhor_resultado) 

**Cross Validation**

Nessa etapa, vamos avaliar a performance de cada algoritmo, tendo como base diferentes bases de treinamento. No caso, vamos dividir a base de dados em 8 partes, uma delas será usada como base de teste e, todas as partes serão usadas, alguma hora como base de testes. Esse procedimento será repedido 20 vezes, a cada ciclo, a base de treinamento e a base de teste é embaralhada. Assim, podemos ter uma noção da variância da performance do algoritmo para a base de dados e também se há overfitting. 

In [47]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression

 
# resultados_random_forest = []
# resultados_knn = []
# resultados_svm = []
# resultados_redes = []
#resultados_regressao = []

for i in range(20):
    kfold = KFold(n_splits = 8, random_state = i, shuffle = True)
    
# #1-------------------Random Forest
#     random_forest = RandomForestClassifier(criterion= 'entropy', min_samples_leaf = 1, min_samples_split = 7, n_estimators = 21)
#     scores = cross_val_score(random_forest, x_data,y_data,cv=kfold)
#     resultados_random_forest.append(scores.mean())
# #2------------------KNN
#     knn = KNeighborsClassifier(n_neighbors= 5, p= 1, weights= 'distance')
#     scores = cross_val_score(knn, x_data,y_data,cv=kfold)
#     resultados_knn.append(scores.mean())
# #3------------------SVM
#     svm = SVC(C =2.4, kernel= 'rbf', tol = 0.001)
#     scores = cross_val_score(svm, x_data,y_data,cv=kfold)
#     resultados_svm.append(scores.mean()) 
# #4------------------Redes
#     redes = MLPClassifier(activation = 'relu', batch_size = 10, hidden_layer_sizes = (15, 15), max_iter = 1500, solver = 'adam')
#     scores = cross_val_score(redes, x_data,y_data,cv=kfold)
#     resultados_redes.append(scores.mean())
#5------------------Regressão Logistica
#     regressao = LogisticRegression(random_state=1)
#     scores = cross_val_score(regressao, x_data,y_data,cv=kfold)
#     resultados_regressao.append(scores.mean())

**Análise Estatística do Algoritmo**

Vemos abaixo os resultados finais da técnica de Cross Validation. Percebemos que o algoritmo SVM produz os melhores scores e além disso possui o menor desvio padrão, significando que o algoritmo é constante, independente da base de dados que ele receber. 

In [48]:
# np.mean(resultados_random_forest), np.mean(resultados_knn), np.mean(resultados_svm), np.mean(resultados_redes), np.mean(resultados_regressao)

In [49]:
# np.std(resultados_random_forest), np.std(resultados_knn), np.std(resultados_svm), np.std(resultados_redes), np.std(resultados_regressao)

In [50]:
#------DIVISAO DA BASE EM TREINAMENTO E TESTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
 
x_data_treinamento, x_data_teste, y_data_treinamento, y_data_teste = train_test_split(x_data, 
                                                                                      y_data, test_size=0.25, random_state=32) 
 
 
 
#---------Treinamento/Teste---------
 
#------Random Forest
random_forest = RandomForestClassifier(criterion= 'entropy', min_samples_leaf = 1, min_samples_split = 7, n_estimators = 21)
random_forest.fit(x_data_treinamento,y_data_treinamento)
previsoes_random = random_forest.predict(x_data_teste)
print('Random:',accuracy_score(y_data_teste,previsoes_random))
prec_random = confusion_matrix(y_data_teste,previsoes_random)
print (classification_report(y_data_teste, previsoes_random))
print(prec_random)
#------KNN
knn = KNeighborsClassifier(n_neighbors= 5, p= 1, weights= 'distance')
knn.fit(x_data_treinamento,y_data_treinamento)
previsoes_knn = knn.predict(x_data_teste)
print('\n KNN',accuracy_score(y_data_teste,previsoes_knn))
prec_knn =confusion_matrix(y_data_teste,previsoes_knn)
print (classification_report(y_data_teste, previsoes_knn))
print(prec_knn)
#------SVM
svm = SVC(C =2.4, kernel= 'rbf', tol = 0.001)
svm.fit(x_data_treinamento,y_data_treinamento)
previsoes_svm = svm.predict(x_data_teste)
print('\n SVM',accuracy_score(y_data_teste,previsoes_svm))
prec_svm =confusion_matrix(y_data_teste,previsoes_svm)
print (classification_report(y_data_teste, previsoes_svm))
print(prec_svm)
#------Redes
rede = MLPClassifier(activation = 'relu', batch_size = 10, hidden_layer_sizes = (15, 15), max_iter = 1500, solver = 'adam')
rede.fit(x_data_treinamento,y_data_treinamento)
previsoes_rede = rede.predict(x_data_teste)
print('\n Rede',accuracy_score(y_data_teste,previsoes_rede))
prec_rede =confusion_matrix(y_data_teste,previsoes_rede)
print (classification_report(y_data_teste, previsoes_rede))
print(prec_rede)
#------Regressão Logistica
from sklearn.linear_model import LogisticRegression
regressao = LogisticRegression(random_state=1)
regressao.fit(x_data_treinamento,y_data_treinamento)
previsoes_regressao = regressao.predict(x_data_teste)
print('\n Regressão',accuracy_score(y_data_teste,previsoes_regressao))
prec_regressao = confusion_matrix(y_data_teste,previsoes_regressao)
print (classification_report(y_data_teste, previsoes_regressao))
print(prec_regressao)

print('/n',dataset.groupby(['recozimento']).size())

Random: 0.8863636363636364
              precision    recall  f1-score   support

           0       1.00      0.91      0.96        47
           1       1.00      0.59      0.74        39
           2       0.82      1.00      0.90        90

    accuracy                           0.89       176
   macro avg       0.94      0.83      0.87       176
weighted avg       0.91      0.89      0.88       176

[[43  0  4]
 [ 0 23 16]
 [ 0  0 90]]

 KNN 0.8579545454545454
              precision    recall  f1-score   support

           0       0.93      0.87      0.90        47
           1       0.85      0.56      0.68        39
           2       0.83      0.98      0.90        90

    accuracy                           0.86       176
   macro avg       0.87      0.80      0.83       176
weighted avg       0.86      0.86      0.85       176

[[41  2  4]
 [ 3 22 14]
 [ 0  2 88]]

 SVM 0.8977272727272727
              precision    recall  f1-score   support

           0       1.00      0.9

Temos acima a performance de cada um dos algoritmos usados. O objetivo estipulado para esse trabalho é obter o menor numero de peças que tem recozimento 'ruim' e serem classificadas como peças que tem recozimento 'ideal' e também obter o menor numero de peças que tem recozimento 'ideal' e serem classificadas como peças que tem recozimento 'ruim'. Tendo em vista essa ideia, foram rodados 4 testes para verificar o comportamento de cada algoritmo para classificar cada uma das classes corretamente. Foram geradas assim as tabelas abaixo:  

![image.png](attachment:image.png)

![image-2.png](attachment:image-2.png)

![image-3.png](attachment:image-3.png)

![image-4.png](attachment:image-4.png)

![image-5.png](attachment:image-5.png)

Para cada tipo de recozimento, foi escolhido o algoritmo mais preciso, que no caso foram:

**Ideal** Random Forest

**Mediano** Random Forest

**Ruim** KNN

Implementação da tomada de decisão em conjunto

In [51]:
prev_final = []

for i in range(y_data_teste.shape[0]):
    if previsoes_random[i] == 1:
        prev_final.append(1)
    elif previsoes_knn[i] == 2:
        prev_final.append(2)
    elif previsoes_random[i] == 0:
        prev_final.append(0)
    else:
        prev_final.append(previsoes_svm[i])

In [52]:
print('\n Regressão',accuracy_score(y_data_teste,prev_final))
prec_final = confusion_matrix(y_data_teste,prev_final)
print (classification_report(y_data_teste, prev_final))
print(prec_final)


 Regressão 0.8863636363636364
              precision    recall  f1-score   support

           0       1.00      0.91      0.96        47
           1       1.00      0.59      0.74        39
           2       0.82      1.00      0.90        90

    accuracy                           0.89       176
   macro avg       0.94      0.83      0.87       176
weighted avg       0.91      0.89      0.88       176

[[43  0  4]
 [ 0 23 16]
 [ 0  0 90]]


**Conclusão**

Ao utilizar os algoritmos em conjunto, foi possível melhorar ainda mais a acertividade do processo de classificação, chegando em 91%. 
A precisão da previsão de recozimentos 'ideais' chegou a 98%, enquanto que a precisão da previsão de recozimentos 'ruins' chegou a 89%. 


**Treinamento Final e Salvamento do Algoritmo**

Vamos agora realizar o treinamento final dos algoritmos usados, agora com a base de dados total e ao fim do treinamento, vamos realizar o salvamento para este poder ser utilizado em uma ferramenta de classificação de recozimento.

In [53]:
import pickle

random_forest = RandomForestClassifier(criterion= 'entropy', min_samples_leaf = 1, min_samples_split = 7, n_estimators = 21)
random_forest.fit(x_data,y_data)
#------KNN
knn = KNeighborsClassifier(n_neighbors= 5, p= 1, weights= 'distance')
knn.fit(x_data,y_data)
#------SVM
svm = SVC(C =2.4, kernel= 'rbf', tol = 0.001)
svm.fit(x_data,y_data)

#------Salvamento do Algoritmo
pickle.dump(random_forest,open('random_finalizado.sav','wb'))
pickle.dump(knn,open('knn_finalizado.sav','wb'))
pickle.dump(svm,open('svm_finalizado.sav','wb'))